In [ ]:
!pip show transformers trl accelerate peft

Разметка

In [ ]:
import re
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
import re

def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = text.replace("ё", "е")
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"\d+", " <NUM> ", text)
    text = re.sub(r"[^a-zа-я0-9\s!?]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

candidate_labels = [
    "бытовая техника",
    "обувь",
    "одежда",
    "посуда",
    "текстиль",
    "товары для детей",
    "украшения и аксессуары",
    "электроника",
    "нет товара"
]

In [ ]:
train["text"] = train["text"].apply(clean_text)
test["text"] = test["text"].apply(clean_text)

In [ ]:
clf1 = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=device)
clf2 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


In [ ]:
!nvidia-smi

Wed Sep 17 10:05:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             29W /   70W |    3810MiB /  15360MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import math
def classify_ensemble_batches(texts, labels, batch_size=32):
    preds = ["нет товара"] * len(texts)
    non_empty_idx = [i for i, t in enumerate(texts) if isinstance(t, str) and t.strip()]
    non_empty_texts = [texts[i] for i in non_empty_idx]

    n_batches = math.ceil(len(non_empty_texts) / batch_size)

    for b in tqdm(range(n_batches), desc="Разметка батчами"):
        start = b * batch_size
        end = start + batch_size
        batch_texts = non_empty_texts[start:end]

        res1 = clf1(batch_texts, candidate_labels=labels, multi_label=False)
        res2 = clf2(batch_texts, candidate_labels=labels, multi_label=False)

        if isinstance(res1, dict): res1 = [res1]
        if isinstance(res2, dict): res2 = [res2]

        for i_local, (r1, r2) in enumerate(zip(res1, res2)):
            idx = non_empty_idx[start + i_local]
            if r1["labels"][0] == r2["labels"][0]:
                preds[idx] = r1["labels"][0]
            else:
                preds[idx] = r1["labels"][0] if r1["scores"][0] > r2["scores"][0] else r2["labels"][0]

    return preds


train["label"] = classify_ensemble_batches(train["text"].tolist(), candidate_labels, batch_size=32)


train.to_csv("train_labeled.csv", index=False, encoding="utf-8-sig")

print("Разметка ансамблем завершена! Файл train_labeled.csv создан.")

Разметка батчами: 100%|██████████| 57/57 [14:56<00:00, 15.72s/it]

Разметка ансамблем завершена! Файл train_labeled.csv создан.


Балансировка классов

In [ ]:
!pip install deep_translator
!nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
Wed Sep 17 10:21:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             33W /   70W |    3866MiB /  15360MiB |      0%      Default |
|                                         |                   

In [ ]:
import pandas as pd
import random
import nltk
from nltk.corpus import wordnet
from itertools import chain
from tqdm import tqdm
from deep_translator import GoogleTranslator
import torch

nltk.download('wordnet')
nltk.download('omw-1.4')

train = pd.read_csv("/content/train_labeled.csv")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def get_synonym(word):
    try:
        en_word = GoogleTranslator(source='ru', target='en').translate(word)
        synonyms = wordnet.synsets(en_word)
        if not synonyms:
            return word

        lemmas = set(chain.from_iterable([s.lemma_names() for s in synonyms]))
        lemmas = [l.replace("_", " ") for l in lemmas if l.lower() != en_word.lower()]

        if not lemmas:
            return word

        en_syn = random.choice(lemmas)
        ru_syn = GoogleTranslator(source='en', target='ru').translate(en_syn)
        return ru_syn
    except:
        return word

def synonym_replacement(sentence, n=1):
    words = sentence.split()
    if not words:
        return sentence
    for _ in range(n):
        idx = random.randint(0, len(words)-1)
        words[idx] = get_synonym(words[idx])
    return " ".join(words)

def random_deletion(sentence, p=0.2):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    new_words = [w for w in words if random.random() > p]
    return " ".join(new_words) if new_words else random.choice(words)

def random_swap(sentence, n=1):
    words = sentence.split()
    if len(words) < 2:
        return sentence
    for _ in range(n):
        i, j = random.sample(range(len(words)), 2)
        words[i], words[j] = words[j], words[i]
    return " ".join(words)

def back_translate(sentence):
    try:
        en = GoogleTranslator(source='ru', target='en').translate(sentence)
        ru = GoogleTranslator(source='en', target='ru').translate(en)
        return ru
    except:
        return sentence

def augment_text(text):
    choice = random.choice(["synonym", "delete", "swap", "back"])
    if choice == "synonym":
        return synonym_replacement(text)
    elif choice == "delete":
        return random_deletion(text)
    elif choice == "swap":
        return random_swap(text)
    else:
        return back_translate(text)

In [ ]:
def balance_classes(df, label_col="category", text_col="review"):
    class_counts = df[label_col].value_counts()
    target_size = class_counts.max()
    new_rows = []

    for cls, count in class_counts.items():
        deficit = target_size - count
        if deficit > 0:
            samples = df[df[label_col] == cls]
            for _ in tqdm(range(deficit), desc=f"Augmenting {cls}"):
                row = samples.sample(1).iloc[0]
                aug_text = augment_text(row[text_col])
                new_rows.append({text_col: aug_text, label_col: cls})

    return pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

train_balanced = balance_classes(train, label_col="label", text_col="text")

Augmenting электроника: 100%|██████████| 630/630 [01:13<00:00,  8.54it/s]


In [ ]:
train_balanced.to_csv("train_balanced.csv", index=False)
print("Балансировка завершена. Размер:", train_balanced.shape)
print(train_balanced["label"].value_counts())

Балансировка завершена. Размер: (5715, 2)
label
одежда                    635
украшения и аксессуары    635
текстиль                  635
нет товара                635
обувь                     635
посуда                    635
электроника               635
товары для детей          635
бытовая техника           635
Name: count, dtype: int64


Тренировка

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
train = pd.read_csv("/content/train_balanced.csv")
train['text'] = train['text'].fillna("")

In [ ]:
label_encoder = LabelEncoder()
train['label_id'] = label_encoder.fit_transform(train['label'])
num_labels = len(label_encoder.classes_)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train['text'].tolist(),
    train['label_id'].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=train['label_id']
)


In [ ]:
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = ReviewsDataset(train_encodings, train_labels)
val_dataset = ReviewsDataset(val_encodings, val_labels)

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    f1 = f1_score(p.label_ids, preds, average='weighted')
    return {"weighted_f1": f1}


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="weighted_f1",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    disable_tqdm=False
)

# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2000816432.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Weighted F1
1,1.015800,1.007938,0.665571
2,0.595500,0.583057,0.830035
3,0.373200,0.390071,0.878536
4,0.212000,0.365287,0.884837
5,0.162400,0.321380,0.897361
6,0.123100,0.299133,0.913505
7,0.100400,0.299878,0.913376


TrainOutput(global_step=1127, training_loss=0.493329531128891, metrics={'train_runtime': 376.2628, 'train_samples_per_second': 95.68, 'train_steps_per_second': 2.995, 'total_flos': 782504978109624.0, 'train_loss': 0.493329531128891, 'epoch': 7.0})

Валидация

In [ ]:
preds = trainer.predict(val_dataset)
val_preds = np.argmax(preds.predictions, axis=1)
print(classification_report(val_labels, val_preds, target_names=label_encoder.classes_))

                        precision    recall  f1-score   support

       бытовая техника       0.98      1.00      0.99        63
            нет товара       0.88      0.79      0.83        63
                 обувь       0.82      0.91      0.86        64
                одежда       0.88      0.80      0.84        64
                посуда       0.95      0.95      0.95        64
              текстиль       0.84      0.83      0.83        63
      товары для детей       0.94      0.97      0.95        64
украшения и аксессуары       0.94      0.98      0.96        63
           электроника       1.00      1.00      1.00        64

              accuracy                           0.91       572
             macro avg       0.91      0.91      0.91       572
          weighted avg       0.91      0.91      0.91       572



Сохранение модели

In [ ]:
output_dir = "my_pretrainedmodel"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('my_pretrainedmodel/tokenizer_config.json',
 'my_pretrainedmodel/special_tokens_map.json',
 'my_pretrainedmodel/vocab.txt',
 'my_pretrainedmodel/added_tokens.json',
 'my_pretrainedmodel/tokenizer.json')

Пример загрузки pretrained модели

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [ ]:
!nvidia-smi

Tue Sep 16 10:57:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   74C    P0             33W /   70W |    6464MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Test

In [ ]:
import torch
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

def predict_in_batches(model, tokenizer, texts, batch_size=32, max_length=256, device="cuda", measure_time=True):
    model.to(device)
    model.eval()

    all_preds = []
    per_sample_times = []

    start_time = time.time() if measure_time else None

    for i in tqdm(range(0, len(texts), batch_size), desc="Inference"):
        batch_texts = texts[i:i+batch_size]

        t0 = time.time()
        encodings = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        encodings = {k: v.to(device) for k, v in encodings.items()}

        with torch.no_grad():
            outputs = model(**encodings)
            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            all_preds.extend(preds)
        t1 = time.time()

        batch_time = (t1 - t0) / len(batch_texts)
        per_sample_times.extend([batch_time] * len(batch_texts))

        del encodings, outputs
        torch.cuda.empty_cache()

    if measure_time:
        total_time = time.time() - start_time
        avg_time = total_time / len(texts)
        max_time = max(per_sample_times)
        print(f"\nСреднее время классификации: {avg_time:.4f} сек/пример")
        print(f"Максимальное время классификации: {max_time:.4f} сек/пример")
        return np.array(all_preds), avg_time, max_time

test = pd.read_csv("/content/test.csv")
test['text'] = test['text'].fillna("")
test_texts = test['text'].tolist()

predictions, avg_time, max_time = predict_in_batches(model, tokenizer, test_texts, batch_size=32)

print("Инференс завершен. Размер предсказаний:", predictions.shape)

In [ ]:
submission = pd.DataFrame({
    "category": label_encoder.inverse_transform(predictions)
})

submission.to_csv("submission.csv", index=False, encoding="utf-8-sig")
print("Готов файл submission.csv с предсказанными категориями в нужном формате.")

Готов файл submission.csv с предсказанными категориями в нужном формате.


In [ ]:
import shutil
shutil.move("/content/my_pretrainedmodel", "/content/drive/MyDrive/my_pretrainedmodel")

'/content/drive/MyDrive/my_pretrainedmodel'